In [1]:
#Khai báo các thư viện cần thiết
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
#Link tới trang web mua bán ô tô ở chợ tốt xét theo các page
website='https://xe.chotot.com/mua-ban-oto?page='

In [3]:
href_list = []
partner_list = []
# Chạy vòng lặp để gắn các số vào trong đường link web site -> thu thập dữ liệu mỗi link
for i in range(1, 2):
    # Gắn số vào link
    url = website + str(i)
    print(url)
    # Tạo một request đến url đó để truy cập web
    response = requests.get(url)

    # Kiểm tra truy cập thành công hay không
    if response.status_code == 200:
        # Tách phần html format và đưa vào lưu trữ trong biến soup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Dò web site thì thấy các tin thường được đăng dưới tag <li> với class là AdItem_wrapperAdItem__S6qPH
        li_tags = soup.find_all('li', class_="AdItem_wrapperAdItem__S6qPH")

        # Chạy để lấy thông tin từng thẻ <li>
        for li_tag in li_tags:
            # Các đường dẫn vào chi tiết của chiếc xe sẽ nằm trong thẻ a với class sau
            a_tag = li_tag.find('a', class_="AdItem_adItem__gDDQT")

            # Rút trích đường link dẫn tới page chi tiết xe
            href = a_tag.get('href') if a_tag else None
            href_list.append(href)

            # Tìm kiếm thẻ đối tác trong từng thẻ <li>
            div_protected_label = li_tag.find('div', class_="AdBody_protectedLabelInTitle__HQgj6")

            # Nếu có là đối tác thì điền "Yes" còn không thì "No"
            partner = "Yes" if div_protected_label else "No"
            partner_list.append(partner)
    else:
        print(f"Failed to retrieve the webpage: {url}")

    

https://xe.chotot.com/mua-ban-oto?page=1


In [4]:
# Tạo dataframe để lưu trữ href và Đối tác
df = pd.DataFrame({'href': href_list, 'Đối tác': partner_list})

In [5]:
#Lưu trữ dataframe
df.to_csv('CrawlLink.csv', index=False)

In [6]:
print(df)

                                                 href Đối tác
0   /mua-ban-oto-quan-go-vap-tp-ho-chi-minh/105578...     Yes
1   /mua-ban-oto-quan-go-vap-tp-ho-chi-minh/109687...     Yes
2   /mua-ban-oto-quan-tan-phu-tp-ho-chi-minh/11036...      No
3   /mua-ban-oto-quan-tan-phu-tp-ho-chi-minh/11048...      No
4   /mua-ban-oto-quan-tan-phu-tp-ho-chi-minh/11036...      No
5    /mua-ban-oto-quan-6-tp-ho-chi-minh/109072076.htm      No
6   /mua-ban-oto-huyen-hoa-vang-da-nang/110576332.htm      No
7    /mua-ban-oto-quan-cai-rang-can-tho/109434668.htm      No
8    /mua-ban-oto-quan-7-tp-ho-chi-minh/110175589.htm     Yes
9   /mua-ban-oto-quan-thanh-xuan-ha-noi/110403137.htm      No
10  /mua-ban-oto-quan-nam-tu-liem-ha-noi/109540180...     Yes
11  /mua-ban-oto-quan-bac-tu-liem-ha-noi/109198959...     Yes
12  /mua-ban-oto-quan-ninh-kieu-can-tho/109299099.htm      No
13   /mua-ban-oto-quan-long-bien-ha-noi/110576235.htm      No
14  /mua-ban-oto-thanh-pho-thu-duc-tp-ho-chi-minh/...      No
15  /mua

In [7]:
# Sau khi thu thập các href thì ấn vô ko truy cập được do thiếu nguồn của đường link nên ta sẽ thêm vô đầu mỗi href
df = pd.read_csv('CrawlLink.csv')
def add_base_url(href):
    return 'https://xe.chotot.com' + href

# Apply the function to the "href" column to add the base URL
df['href'] = df['href'].apply(add_base_url)
df.to_csv('FullLink.csv', index=False)

In [8]:
df['href'][0]

'https://xe.chotot.com/mua-ban-oto-quan-go-vap-tp-ho-chi-minh/105578504.htm#px=SR-stickyad-[PO-1][PL-top]'

In [9]:
df= pd.read_csv('./FullLink.csv')

In [10]:
df.head(5)

,href,Đối tác
0,https://xe.chotot.com/mua-ban-oto-quan-go-vap-...,Yes
1,https://xe.chotot.com/mua-ban-oto-quan-go-vap-...,Yes
2,https://xe.chotot.com/mua-ban-oto-quan-tan-phu...,No
3,https://xe.chotot.com/mua-ban-oto-quan-tan-phu...,No
4,https://xe.chotot.com/mua-ban-oto-quan-tan-phu...,No


In [11]:
column_names = [
    'Tiêu đề',
    'Chủ',
    'Phân loại người bán',
    'Địa chỉ',
    'Hãng',
    'Dòng xe',
    'Năm sản xuất',
    'Số Km đã đi',
    'Tình trạng',
    'Hộp số',
    'Nhiên liệu',
    'Kiểu dáng',
    'Số chỗ',
    'Xuất xứ',
    'Chính sách bảo hành',
    'Trọng lượng',
    'Trọng tải',
    'Giá'
    ]

In [12]:
def extract_data_with_a_tag(soup, itemprop):
    element = soup.find('a', itemprop=itemprop)
    return element.get_text(strip=True) if element else '-'

def extract_data(soup, itemprop):
    element = soup.find('span', itemprop=itemprop)    
    return element.get_text(strip=True) if element else '-'
    
def click_button(driver, class_name):
    button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, class_name)))
    button.click()

In [13]:
dfs =[]
count=0

for url in df['href']:
    print("working on url: ", count)
    driver = webdriver.Firefox()
    driver.get(url)
    count+=1
    try:
        # Chờ và nhấn nút để đóng popup
        click_button(driver, "aw__s13upwxx")
        click_button(driver, "styles_button__SVZnw")
        
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        location_element = soup.find('span', class_='fz13') 
        location_text = location_element.get_text(strip=True) if location_element else '-'

        owner_element = soup.find('b', role='presentation') 
        owner_text = owner_element.get_text(strip=True) if owner_element else '-'

        seller_element = soup.find('div', class_='SellerProfile_statusOnlineDivVeh__kgmx4') 
        seller_text = seller_element.get_text(strip=True) if seller_element else '-'

        frist_div = soup.find('div', class_='DetailViewAB_adviewItem__20YCA')
        if frist_div:
            # Tìm thẻ span với thuộc tính itemprop="price" bên trong div_element
            name_element = frist_div.find('h1', itemprop='name', class_='AdDecriptionVeh_adTitle__vEuKD')
            name_text = name_element.get_text(strip=True) if name_element else '-'
                
            price_element = frist_div.find('span', itemprop='price')
            price_text = price_element.get_text(strip=True) if price_element else '-'

        else:
            print("Không tìm thấy div với class 'DetailViewAB_adviewItem__20YCA'.")
        
        div_element = soup.find('div', class_='AdParam_adParamContainerVeh__Vz4Zt')
        if div_element:
            carbrand_text = extract_data_with_a_tag(div_element, 'carbrand')
            carmodel_text = extract_data_with_a_tag(div_element, 'carmodel')
            Prodate_text = extract_data(div_element, 'mfdate')
            KM_text = extract_data(div_element, 'mileage_v2')
            Condition_text = extract_data(div_element, 'condition_ad')
            Gear_text = extract_data(div_element, 'gearbox')
            Fuel_text = extract_data(div_element, 'fuel')
            cartype_text = extract_data(div_element, 'cartype')
            carseats_text = extract_data(div_element, 'carseats')
            carorigin_text = extract_data(div_element, 'carorigin')
            veh_warranty_policy_text= extract_data(div_element, 'veh_warranty_policy')
            veh_unladen_weight_text= extract_data(div_element, 'veh_unladen_weight')
            veh_gross_weight_text = extract_data(div_element, 'veh_gross_weight')
            
            extract_data_list = [name_text,owner_text,seller_text,location_text,carbrand_text,carmodel_text,Prodate_text,KM_text,Condition_text,
                        Gear_text,Fuel_text,cartype_text,carseats_text,carorigin_text,
                        veh_warranty_policy_text,veh_unladen_weight_text,veh_gross_weight_text,price_text]
        else:
            print("Không tìm thấy div với class 'AdParam_adParamContainerVeh__Vz4Zt'.")
            
    except Exception as e:
        print(f"Failed to retrieve data from URL: {url}")
        print(f"Error: {str(e)}")
        extract_data_list = ['-','-','-','-','-','-','-','-','-','-','-','-','-','-','-','-','-','-']
    new_df = pd.DataFrame([extract_data_list], columns=column_names)
    dfs.append(new_df)
    driver.quit()
final_df = pd.concat(dfs, ignore_index=True, axis=0)

working on url:  0
working on url:  1
working on url:  2
working on url:  3
working on url:  4
working on url:  5
working on url:  6
working on url:  7
working on url:  8
working on url:  9
working on url:  10
working on url:  11
working on url:  12
working on url:  13
working on url:  14
working on url:  15
Failed to retrieve data from URL: https://xe.chotot.com/mua-ban-oto-quan-thanh-xuan-ha-noi/110457787.htm
Error: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16

working on url:  16
working on url:  17
working on url:  18
working on url:  19
working on url:  20
Failed to retrieve data from URL: https://xe.chotot.com/mua-ban-oto-quan-1-tp-ho-chi-minh/103502712.htm
Error: Message: unexpected end of hex escape at line 1 column 1

In [14]:
final_df

,Tiêu đề,Chủ,Phân loại người bán,Địa chỉ,Hãng,Dòng xe,Năm sản xuất,Số Km đã đi,Tình trạng,Hộp số,Nhiên liệu,Kiểu dáng,Số chỗ,Xuất xứ,Chính sách bảo hành,Trọng lượng,Trọng tải,Giá
0,Bán mazda6 2.5 - T1/2015,DH AUTO,-,"Phường 5, Quận Gò Vấp, Tp Hồ Chí Minh",Mazda,6,2015,80000,Đã sử dụng,Tự động,Xăng,Sedan,5,Việt Nam,Bảo hành hãng,> 1 tấn,> 2 tấn,420.000.000 đ
1,Bán Kia K3 1.6 AT sx 2014 1 đời chủ 109k km,DH AUTO,-,"193 Đường Số, Phường 5, Quận Gò Vấp, Tp Hồ Chí...",Kia,K3,2014,109000,Đã sử dụng,Tự động,Xăng,-,5,Hàn Quốc,Bảo hành hãng,> 1 tấn,> 2 tấn,358.000.000 đ
2,𝗨̛𝗨 Đ𝗔̃𝗜 𝗛𝗢̛𝗡 𝟮𝟴𝟬 𝗧𝗥𝗜𝗘̣̂𝗨 𝗞𝗛𝗜 𝗠𝗨𝗔 𝗫𝗘 𝗦-𝗖𝗹𝗮𝘀𝘀,Mercedes-Benz Vietnam Star Trường Chinh,-,"32 Đường Tân Thắng, Phường Tân Sơn Nhì, Quận T...",Mercedes Benz,S Class,2023,-,Mới,Tự động,Xăng,Sedan,4,Thái Lan,Bảo hành hãng,> 1 tấn,> 2 tấn,5.039.000.000 đ
3,"BÁN XE FORTUNER MÁY DẦU 2 CẦU, XE NHẬP INDO",Nguyen Julia,Cá nhân,"Lê Văn Phan, Phường Phú Thọ Hòa, Quận Tân Phú,...",Toyota,Fortuner,2019,28000,Đã sử dụng,Tự động,Dầu,SUV / Cross over,7,Nước khác,Bảo hành hãng,> 1 tấn,> 2 tấn,980.000.000 đ
4,𝐍𝐄𝐖 𝐆𝐋𝐂 𝟐𝟎𝟐𝟑 - Ưu Đãi 100% Trước Bạ & BHVC ....,Mercedes-Benz Vietnam Star Trường Chinh,-,"32 Đường Tân Thắng, Phường Tân Sơn Nhì, Quận T...",Mercedes Benz,GLC Class,2023,-,Mới,Tự động,Xăng,SUV / Cross over,5,Việt Nam,Bảo hành hãng,> 1 tấn,> 2 tấn,2.299.000.000 đ
5,Tucson turbo giá 899.000.000,Ngọc Hyundai,Bán chuyên,"Đường Phạm Đình Hổ, Phường 2, Quận 6, Tp Hồ Ch...",Hyundai,Tucson,2023,-,Mới,Tự động,Xăng,SUV / Cross over,5,Hàn Quốc,Bảo hành hãng,> 1 tấn,> 2 tấn,899.000.000 đ
6,Bán MG5 Bản Đầy Đủ - Số Tự Động. 450 Triệu,Tâm Xe Lướt Đà Nẵng,Cá nhân,"Xã Hòa Châu, Huyện Hòa Vang, Đà Nẵng",MG,5,2022,310000,Đã sử dụng,Tự động,Xăng,Sedan,-,Thái Lan,Bảo hành hãng,> 1 tấn,> 2 tấn,455.000.000 đ
7,camry 2007 nhập mỹ xe rin không lổi,MINH HÙNG,Bán chuyên,"Chợ Cái Chanh, Đường Tỉnh 925, Phường Thường T...",Toyota,Camry,2008,95000,Đã sử dụng,Tự động,Xăng,Sedan,4,Mỹ,Bảo hành hãng,> 1 tấn,> 2 tấn,425.000.000 đ
8,Peugeot 3008 Allure Sản Xuất 2021 Cực Mới,LONG ANH AUTO 6668,-,"66-68 Hẻm 8 Đường Số 7, Phường Tân Kiểng, Quận...",Peugeot,3008,2021,37000,Đã sử dụng,Tự động,Xăng,SUV / Cross over,5,Việt Nam,Bảo hành hãng,> 1 tấn,> 2 tấn,799.000.000 đ
9,C300 Coupe 2019 Trắng Kem,DSCARS AUTO,-,"Phường Nhân Chính, Quận Thanh Xuân, Hà Nội",Mercedes Benz,C Class,2017,35000,Đã sử dụng,Tự động,Xăng,Coupe (2 cửa),4,Đức,Bảo hành hãng,> 1 tấn,> 2 tấn,1.550.000.000 đ


In [15]:

final_df['Đối tác'] = df['Đối tác']
final_df['href'] = df['href']
final_df.to_csv('./CarPricePrediction_full.csv', index=False)